# ABS Quarterly Building Activity 8752

## Python set-up

In [1]:
# system imports
import textwrap

# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from plotting import seas_trend_plot, line_plot


# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# Display charts in the notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("8752.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

Table 87520DO001_202403 has no 'Index' sheet.
Table 87520DO002_202403 has no 'Index' sheet.
Table 87520DO003_202403 has no 'Index' sheet.
Table 87520DO004_202403 has no 'Index' sheet.
Table 87520DO005_202403 has no 'Index' sheet.
Table 87520DO006_202403 has no 'Index' sheet.
Table 87520_activity has no 'Index' sheet.


## Plot

### Headline charts

#### Core code

Note: generates some "Nothing selected" messages for the under construction block.

In [3]:
def plot_headline() -> None:
    """Plot headline series for the building activity data."""
    # pylint: disable=line-too-long

    wanted = [
        # table, did
        ["87520033", "Dwelling units commenced ;  Total Sectors ;  Houses ;  New ;"],
        [
            "87520033",
            "Dwelling units commenced ;  Total Sectors ;  Total Other Residential ;  New ;",
        ],
        [
            "87520033",
            "Dwelling units commenced ;  Total Sectors ;  Total (Type of Building) ;  Total (Type of Work) ;",
        ],
        ["87520037", "Dwelling units completed ;  Total Sectors ;  Houses ;  New ;"],
        [
            "87520037",
            "Dwelling units completed ;  Total Sectors ;  Total Other Residential ;  New ;",
        ],
        [
            "87520037",
            "Dwelling units completed ;  Total Sectors ;  Total (Type of Building) ;  Total (Type of Work) ;",
        ],
        # this next block will generate "Nothing selected" errors (as there is no SeasAdj or Trend data for these series)
        [
            "87520076",
            "Dwelling units under construction ;  Total Sectors ;  Houses ;  New ;",
        ],
        [
            "87520076",
            "Dwelling units under construction ;  Total Sectors ;  Total Other Residential ;  New ;",
        ],
        [
            "87520076",
            "Dwelling units under construction ;  Total Sectors ;  Dwellings excluding new residential ;  Total (Type of Work) ;",
        ],
        [
            "87520076",
            "Dwelling units under construction ;  Total Sectors ;  Total (Type of Building) ;  Total (Type of Work) ;",
        ],
        [
            "87520080",
            "Dwelling units not yet commenced ;  Houses ;  New ;  Australia ;",
        ],
        [
            "87520080",
            "Dwelling units not yet commenced ;  Total Other Residential ;  New ;  Australia ;",
        ],
        [
            "87520080",
            "Dwelling units not yet commenced ;  Total (Type of Building) ;  Total (Type of Work) ;  Australia ;",
        ],
    ]

    for table, did in wanted:
        f_dict = {}
        for stype in "Seasonally Adjusted", "Trend", "Original":
            search = {
                table: mc.table,
                did: mc.did,
                stype: mc.stype,
            }
            try:
                _t, sid, u = ra.find_abs_id(meta, search)
                f_dict[stype] = abs_dict[table][sid]
            except ValueError:
                continue
        found = pd.DataFrame(f_dict).sort_index()
        found, units = ra.recalibrate(found, u)

        if "Seasonally Adjusted" in f_dict or "Trend" in f_dict:
            if "Original" in f_dict:
                found = found.drop("Original", axis=1)
            seas_trend_plot(
                found,
                starts=plot_times,
                tags=plot_tags,
                title=textwrap.fill(did, 60),
                lfooter="Australia. ",
                rfooter=f"ABS: {table[:4]}",
                ylabel=f"{units}/Quarter",
                show=SHOW,
            )

        elif "Original" in f_dict:
            line_plot(
                found["Original"],
                starts=plot_times,
                tags=plot_tags,
                title=textwrap.fill(did, 60),
                lfooter="Australia. ",
                rfooter=f"ABS: {table[:4]}",
                ylabel=f"{units}/Quarter",
                legend={"loc": "best", "fontsize": "small"},
                width=2,
                show=SHOW,
            )


plot_headline()

Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?
Nothing selected?


## building cycle

In [4]:
def building_cycle():
    """plot the building cycle."""
    # pylint: disable=line-too-long

    wanted = (
        (
            "87520033",
            "Dwelling units commenced ;  Total Sectors ;  Total (Type of Building) ;  Total (Type of Work) ;",
        ),
        (
            "87520037",
            "Dwelling units completed ;  Total Sectors ;  Total (Type of Building) ;  Total (Type of Work) ;",
        ),
    )

    common_line_plot = {
        # common arguments for the line_plot function
        "axhline": {
            "y": 60,
            "label": "Target: build 1.2 million well-located new homes over 5 years, from 1 July 2024",
            "color": "darkred",
            "linestyle": "--",
            "lw": 0.75,
        },
        "width": 2,
        "show": SHOW,
        "legend": {"loc": "best", "fontsize": "x-small"},
        "lfooter": "Australia. ",
        "starts": plot_times,
        "tags": plot_tags,
    }

    f_dict = {}
    for table, did in wanted:
        search = {
            table: mc.table,
            did: mc.did,
            "Trend": mc.stype,
        }
        _table, sid, units = ra.find_abs_id(meta, search)
        f_dict[did] = abs_dict[table][sid]
    found1 = pd.DataFrame(f_dict).sort_index()

    found1, units = ra.recalibrate(found1, units)
    line_plot(
        found1,
        title="Trend Commencements and Completions",
        rfooter=f"ABS: {table[:4]}",
        ylabel=f"{units}/Quarter",
        **common_line_plot,
    )

    # now add in the trend building approvals on a quarterly sum basis
    cat, sid = "8731.0", "A422072L"  # trend approvals, all sectors
    ta, ta_meta = ra.read_abs_series("8731.0", sid, single_excel_only="8731006")
    f_dict["Building Approvals ;  " + ta_meta.loc[sid, mc.did]] = ra.monthly_to_qtly(
        ta["A422072L"], f="sum"
    )
    found2 = pd.DataFrame(f_dict).sort_index()
    found2, units = ra.recalibrate(found2, ta_meta.loc[sid, mc.unit])
    line_plot(
        found2,
        title="Trend Approvals, Commencements and Completions",
        rfooter=f"ABS: {table[:4]} {cat[:4]}",
        ylabel=f"{units}/Quarter",
        # style=["--", "-", "-."],
        color=["cornflowerblue", "navy", "darkorange"],
        **common_line_plot,
    )


building_cycle()

## Finished

In [5]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Aug 01 2024 13:39:09

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

readabs: 0.0.17
pandas : 2.2.2

Watermark: 2.4.3



In [6]:
print("Finished")

Finished
